In [1]:
# Project baby otter 
# this otter collects promotions from ssg.com!

In [3]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import os 
from config import Config

In [64]:
url = 'https://www.ssg.com/' 
r = requests.get(url)

if r.status_code == 200:
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

else : 
    print(r.status_code)

In [18]:
events = soup.find_all(class_ = 'ssghero_slide_col')
event_ls = [] 
for event in events : 
    try : 
        link = event.find(href=True)["href"]
        if "nevntId" in link : 
            event_ls.append(event)
        # 확실하게 event인 거만 뽑아내기 (프로모션 없는 단순 제품 나열형 기획전 제외 -> 코드가 다름) 
    except : 
        continue

In [25]:
df = pd.DataFrame()
for event in event_ls : 
    
    #기본정보
    titlemain = event.find(class_ = 'ssghero_titmain').text
    titlesub = event.find(class_ = 'ssghero_titsub').text
    link = link = event.find(href=True)["href"]

    row = pd.DataFrame.from_dict( [{"titlemain" : titlemain
                        ,"titlesub" : titlesub
                        ,"link" : link
                        }])
    df = pd.concat([df, row], ignore_index = True)

In [26]:
df

,titlemain,titlesub,link
0,명절 선물도 이마트몰 SSG-배송,2022 추석선물 지금이 알뜰 구매찬스!,https://event.ssg.com/eventDetail.ssg?nevntId=...
1,트레이더스 SSG-배송으로 선물 하세요,추석 이후에도 감사의 마음은 그대로!,https://event.ssg.com/eventDetail.ssg?nevntId=...
2,대한민국 물가 해결사! 긴급출동 오반장,장보기 특가부터 추석선물까지 한번에,https://event.ssg.com/eventDetail.ssg?nevntId=...
3,선물엔 뷰티,"럭셔리&트랜드뷰티 ~20%쿠폰, SSG머니 적립 혜택까지!",https://event.ssg.com/eventDetail.ssg?nevntId=...
4,9월 여행 특가 위크,가을 맞이 여행 패키지 상품 ~2만원 즉시 할인,https://event.ssg.com/eventDetail.ssg?nevntId=...
5,쓱배송은 달립니다.,쓱배송 응원하면 SSG머니의 행운이,https://event.ssg.com/eventDetail.ssg?nevntId=...
6,2022 대한민국 수산대전,추석 성수품 농축수산물 할인대전,https://event.ssg.com/eventDetail.ssg?nevntId=...
7,쇼핑을 넘어 일상의 혜택까지,스마일클럽은 TMAP 주차 할인,https://event.ssg.com/eventDetail.ssg?nevntId=...
8,9월의 스마일클럽 웰컴 기프트 5천원,카테고리 선착순 할인쿠폰도 놓치지 말아요,https://event.ssg.com/eventDetail.ssg?nevntId=...
9,아직 늦지않은 추석 명절 GIFT,#배송마감 #퀵배송 #마지막 명절열차,https://event.ssg.com/eventDetail.ssg?nevntId=...


In [69]:
sub_url = 'https://event.ssg.com/m/eventDetail.ssg?nevntId=1000000001965'
r2 = requests.get(sub_url)

if r2.status_code == 200:
    html = r2.text
    soup = BeautifulSoup(html, 'html.parser')

else : 
    print(r2.status_code)

In [71]:
#1. 날짜

evt_title = soup.find_all(class_ = 'etxt')

evt_date = soup.find_all(class_ = 'edays')
print(evt_title)
print(evt_date)

[<h2 class="etxt">(0912-18) 9월 쇼핑 익스프레스 WEEK</h2>]
[<p class="edays">
<span>기간</span>
<span> | </span>
<span>2022-09-12 ~ 2022-09-18</span>
</p>]


In [72]:
p = re.compile('[0-9]{4}.[0-9]{2}.[0-9]{2}')
p.findall(evt_date[0].text)

['2022-09-12', '2022-09-18']

In [87]:
#cm_mall_ic
evt_mall = soup.find(class_ = 'cm_mall_ic ty_text_s').text
evt_mall = evt_mall.splitlines()
evt_mall = [m for m in evt_mall if m != '']
# mall_ls = []
# #
# for mall in evt_mall : 
#     print(mall.text)
#     print(type(mall.text))
    

In [88]:
evt_mall

['SSG.COM', '신세계몰', '신세계백화점']

In [90]:

test = soup.find_all('span', class_=['evt', 'link_area', 'blind'])

#
for t in test : 
    print(t.text)

#//*[@id="_section7000712982"]/div[2]/ul/li[2]/div/div[2]/div[2]/div[1]/a/div[1]/span[3]
##_section7000712982 > div.cmitem_grid > ul > li:nth-child(2) > div > div.mnsditem_goods > div.mnsditem_detail > div.mnsditem_maininfo > a > div.mnsditem_tit > span.mnsditem_goods_tit

SSG
신세계적 쇼핑포털 SSG.COM
몰 목록 펼치기
몰 목록 접기
SSG.COM
검색
장바구니
함께 장보기
담은 상품 수
이전 페이지
관심상품 취소
관심상품 등록
공유하기
쇼핑열차 출발! ~12% 쿠폰팩
더 강력한 스마일클럽 15% 쿠폰
다시 오지 않을 단 하루! 브랜드데이
쇼핑 정착역 카테고리별 BEST 특가
할인의 완성 카드혜택
레이어팝업 닫기
쿠폰팩 다운로드
쿠폰 사용 전에 꼭 확인하세요(쿠폰 유의사항 레이어 열기)
일부 상품은 쿠폰 적용에서 제외됩니다. (적용 제외 상품일 경우, 결제페이지에서 쿠폰적용이 되지 않습니다.)
7만원 이상 구매시 2만원 할인
쿠폰이 모두 소진되었습니다! 성원에 감사드립니다.
레이어팝업 닫기
쿠폰 다운로드 - 기간내 ID당 1매 다운로드 가능
쿠폰 사용 전에 꼭 확인하세요(쿠폰 유의사항 레이어 열기)
일부 상품은 쿠폰 적용에서 제외됩니다. (적용 제외 상품일 경우, 결제페이지에서 쿠폰적용이 되지 않습니다.)
바로가기
키엘(15%할인) (대한민국 NO.1 국민 수분크림 토너 대용량 샘플 + 울트라 5종 + 파우치 증정) 바로가기
랑콤(15%할인) (5초에 한 병씩 판매되는 제니피끄 세럼 50ml 구매 시 20ml + 클라리피끄 에센스 증정) 바로가기
마시모두띠(적립혜택) (NEW IN 가을 스타일링 제안 & 기간한정 적립 혜택) 바로가기
랩/플라스틱아일랜드/에고이스트(최대 70%할인) (단 일주일! 패밀리세일 사계절 상품 최대 70% OFF) 바로가기
빈폴키즈(최저 30%할인) (SSG EXCLUSIVE 패밀리룩으로 내 아이 예쁘게 입히기) 바로가기
헬로카봇(최대 9%할인) (완구와 애니메이션의 융합으로 아이들에게 즐거움을 주는 장난감) 바로가기
쿠쿠(최대 15%할인) (9월 SSG 단독 런칭특가! IH 마스터셰프 사일런스 밥솥) 바로가기
LG전자(최대 8%할인) (생활가전 인기상품 득템 찬스) 바로가기
LG생활건장(최대 20%할인) (세제/헤어/구강 外 생필품 BEST 특가전) 바로가기
P&G(최대 43%할인) (다우니/페

In [43]:
for i in range(len(test)):
    text_test = [t.text for t in test][i].splitlines()
    print([t for t in text_test if (t != '') & ('바로가기' not in t) and (t != '') & ('쿠폰' in t) and (t != '') & ('%' in t) or (t != '') & ('구매시' in t)])
    #print(text_test)

[]
['쇼핑열차 출발! ~12% 쿠폰팩', '더 강력한 스마일클럽 15% 쿠폰']
['쿠폰팩 받고 쇼핑열차 출발! ~12% 장바구니쿠폰 + 스마일클럽 15% 쿠폰', ' 장바구니 쿠폰 7% 5만원 이상 구매시 최대 5천원 할인', ' 장바구니 쿠폰 10% 7만원 이상 구매시 최대 1만원 할인', ' 장바구니 쿠폰 12% 10만원 이상 구매시 최대 2만원 할인']
['장바구니 쿠폰 15%', '7만원 이상 구매시 2만원 할인']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [45]:
for i in range(len(test)):
    text_test = [t.text for t in test][i].splitlines()
    print([t for t in text_test if (t != '') & ('카드' in t) and (t != '') & ('%' in t) ])
    #print(text_test)

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['(신한카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기', '(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기', '(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기', '(삼성카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기', '(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기', '(KB카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기', '(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기', '(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기', '(현대카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기', '(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기', '(신한카드) [SSGPAY 전용] 7만원 이상 7% 청구할인 (일 5만원 한) 자세히 보기', '(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기', '(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기']
[]
[]
[]
[]


In [53]:
card = soup.find_all(attrs= {'class' : 'timer_cnt'})

#
for c in card : 
    print(c["data-end-date"])


2022/09/13 00:00
2022/09/14 00:00
2022/09/15 00:00
2022/09/16 00:00
2022/09/17 00:00
2022/09/19 00:00
2022/09/13 00:00
2022/09/15 00:00
2022/09/16 00:00
2022/09/17 00:00
2022/09/19 00:00


In [57]:
for c in card: 
    print(c)

<div class="timer_cnt" data-end-date="2022/09/13 00:00" data-start-date="2022/09/01 00:00">
<img alt="" src="https://sui.ssgcdn.com/ui/common/img/promotion/2022/220912_ssgshopping_express/mobile/m_img06_1.jpg"/>
<h5 class="blind">뷰티풀 데이</h5>
<div class="link_area02">
<a href="https://m-shinsegaemall.ssg.com/item/itemView.ssg?itemId=1000042694671"><span class="blind">키엘(15%할인) (대한민국 NO.1 국민 수분크림 토너 대용량 샘플 + 울트라 5종 + 파우치 증정) 바로가기</span></a>
<a href="https://m.ssg.com/plan/planShop.ssg?dispCmptId=6000248617"><span class="blind">랑콤(15%할인) (5초에 한 병씩 판매되는 제니피끄 세럼 50ml 구매 시 20ml + 클라리피끄 에센스 증정) 바로가기</span></a>
<a href="https://m-shinsegaemall.ssg.com/plan/planShop.ssg?dispCmptId=6000221299"><span class="blind">마시모두띠(적립혜택) (NEW IN 가을 스타일링 제안 &amp; 기간한정 적립 혜택) 바로가기</span></a>
<a href="https://m-shinsegaemall.ssg.com/plan/planShop.ssg?planShopId=6000022485"><span class="blind">랩/플라스틱아일랜드/에고이스트(최대 70%할인) (단 일주일! 패밀리세일 사계절 상품 최대 70% OFF) 바로가기</span></a>
</div>
</div>
<div class="timer_cnt" data-en

In [91]:
for i in range(len(card)):
    if "카드" in card[i].text : 
        print(card[i]['data-end-date'])
        print(card[i]['data-start-date'])
        print(card[i].text)

2022/09/13 00:00
2022/09/01 00:00


뷰티풀 데이
(신한카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기
(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기
(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기

2022/09/15 00:00
2022/09/13 00:00


뷰티풀 데이
(삼성카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기
(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기

2022/09/16 00:00
2022/09/15 00:00


뷰티풀 데이
(KB카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기
(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기
(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기

2022/09/17 00:00
2022/09/16 00:00


뷰티풀 데이
(현대카드) [SSGPAY 전용] 7만원 이상 8% 청구할인 (일 5만원 한) 자세히 보기
(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기

2022/09/19 00:00
2022/09/17 00:00


뷰티풀 데이
(신한카드) [SSGPAY 전용] 7만원 이상 7% 청구할인 (일 5만원 한) 자세히 보기
(SSG.COM 카드 Edition2) 5% 선할인 (상품당 3만원 한) 자세히 보기
(SSG.COM 삼성카드) 7% 선할인 (상품당 3만원 한) 자세히 보기



In [37]:
[t for t in text_test if (t != '') & ('%' in t)] # 정규표현식으로 문자 없는 것 뺄 수 있을듯 

['쿠폰팩 받고 쇼핑열차 출발! ~12% 장바구니쿠폰 + 스마일클럽 15% 쿠폰',
 ' 장바구니 쿠폰 7% 5만원 이상 구매시 최대 5천원 할인',
 ' 장바구니 쿠폰 10% 7만원 이상 구매시 최대 1만원 할인',
 ' 장바구니 쿠폰 12% 10만원 이상 구매시 최대 2만원 할인']

In [103]:
prd_list = soup.find_all(class_ = 'cmitem_goods_tit')
for prd in prd_list : 
    print(prd.text)
    

In [87]:
title = soup.find_all(class_ = 'cevent_subject_tit')
title[0].text

'9/15 단하루! 쓱배송데이'

In [89]:
term = soup.find_all(class_ = 'cevent_data_term')
term[0].text

'\n              \t\t2022-09-15\xa0~\xa02022-09-15'

In [92]:
text_ls = soup.find_all(class_ = 'blind')
for text in text_ls : 
    print(text.text)

SSG.COM
몰 목록 펼치기
SSG.COM
몰 목록 접기
검색
상승
상승
상승
상승
하락
상승
상승
상승
상승
상승
상승
상승
상승
상승
하락
상승
상승
상승
상승
상승
상승
상승
상승
상승
상승
상승
상승
상승
상승
상승
좋아요
MY SSG
주문배송조회
장바구니
SSG.COM
이마트몰
트레이더스
새벽배송
페이스북
트위터
관심상품 취소
관심상품 등록

할인에 적립을 더하다 
쓱배송 DAY
9월 15일 목요일, 단 하루!

9월 15일 목요일, 단 하루!
최대 10% 장바구니 쿠폰 +5% 적립 바로보기
가격비교 이제는 끝! 가격의 끝 바로보기
쓱배송 특가상품 최대 50% 할인 바로보기
10% 장바구니 쿠폰
스마일클럽 멤버십 전용 10% 할인쿠폰
이마트몰 쓱배송 / 트레이더스몰 쓱배송 및 점포택배 / 새벽배송 상품
8만원 이상 결제 시 최대 1만원 할인 (ID당 1일 1매 다운)
10% 할인 쿠폰 다운받기


          발급 기간
         

          2022년 9월 15일, 단 하루 (ID당 1일 1회 다운가능)
         

          사용 기간
         

          발급 당일 사용 가능
         

쿠폰 사용 전 꼭 확인하세요(쿠폰 유의사항 레이어 열기)
선착순 쿠폰 Sold out !
SSG 스마일클럽 전용 혜택
9월에는 매일매일 스마일, 스마일클럽 혜택

쓱·새벽·트레이더스 SSG머니 최대 5% 적립. 상품당 최대 1만원, 월 최대 5만원까지 모두 적립해드려요!(월 가입비 3,900원)
15,000원만 담아도 쓱 · 새벽 무료배송 (SSG 가입 고객 전용) 장보기 상품의 무료배송 한도가 쓱-내려가요! (매월 1장+1장 더!)
(9월 한정)매주 월요일마다 장보기 최대 10% 더블적립 DAY 스마일클럽 기본 적립 5% + 월요일마다 추가 적립 5%
(9월 한정)매주 열리는 최대 50% BIG SALE WEEK 이번주는 유아동 + 스포츠 최대 20% 할인 (매주 카테고리별 할인)
(9월 한정)신규고객

In [50]:
img_ls = soup.find_all("img")

img_tmp = [img["src"] for img in img_ls if "promotion" in img["src"]]


In [52]:
img_tmp

['https://sui.ssgcdn.com/ui/common/img/promotion/2022/220905_monday_moon/mobile/m_img01.jpg',
 'https://sui.ssgcdn.com/ui/common/img/promotion/2022/220905_monday_moon/mobile/m_img02.jpg',
 'https://sui.ssgcdn.com/ui/common/img/promotion/2022/220905_monday_moon/mobile/m_img03.jpg',
 'https://sui.ssgcdn.com/ui/common/img/promotion/2022/220905_monday_moon/mobile/m_img04.jpg',
 'https://sui.ssgcdn.com/ui/common/img/promotion/2022/220905_monday_moon/mobile/m_img05.jpg',
 'https://sui.ssgcdn.com/ui/common/img/promotion/2022/220905_monday_moon/mobile/m_img06.jpg']

In [53]:
def detect_text_uri(uri):
    """Detects text in the file located in Google Cloud Storage or on the Web.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()
    image = vision.Image()
    image.source.image_uri = uri

    response = client.text_detection(image=image)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in text.bounding_poly.vertices])

        print('bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [64]:
detect_text_uri(img_tmp[0])

Texts:

"럭셔리뷰티 ~12% 상품쿠폰
적용몰 : 신세계백화점몰, 시코르
적용 카테고리 : 스킨케어, 메이크업, 헤어/바디, 향수
상품쿠폰
12%
선착순
5만장
4만원 이상 상품 구매시 최대 1만 5천원 할인
S P
쿠폰 다운로드 받기
-
- 이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외.
트렌드뷰티 ~7% 상품쿠폰
SSG.COM
상품쿠폰
적용몰 : 신세계몰, 이마트몰
적용 카테고리 : 뷰티 전체, 일상 헤어케어&바디케어
1%
선착순
3만장
SSG.COM
2만원 이상 상품 구매시 최대 5천원 할인
se
쿠폰 다운로드 받기
이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외-"
bounds: (77,38),(673,38),(673,1901),(77,1901)

"럭셔리"
bounds: (179,39),(266,39),(266,73),(179,73)

"뷰티"
bounds: (271,39),(329,39),(329,73),(271,73)

"~"
bounds: (342,39),(364,39),(364,73),(342,73)

"12"
bounds: (365,39),(404,39),(404,73),(365,73)

"%"
bounds: (405,40),(436,40),(436,74),(405,74)

"상품"
bounds: (448,40),(510,40),(510,74),(448,74)

"쿠폰"
bounds: (513,40),(571,40),(571,74),(513,74)

"적용"
bounds: (221,161),(266,161),(266,187),(221,187)

"몰"
bounds: (268,161),(289,161),(289,187),(268,187)

":"
bounds: (298,161),(305,161),(305,187),(298,187)

"신세계"
bounds: (312,161),(378,161),(378,187),(312,187)

"백화점"
bounds: (381,161),(447,161),(447,

In [71]:
texts[0].description

'럭셔리뷰티 ~12% 상품쿠폰\n적용몰 : 신세계백화점몰, 시코르\n적용 카테고리 : 스킨케어, 메이크업, 헤어/바디, 향수\n상품쿠폰\n12%\n선착순\n5만장\n4만원 이상 상품 구매시 최대 1만 5천원 할인\nS P\n쿠폰 다운로드 받기\n-\n- 이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외.\n트렌드뷰티 ~7% 상품쿠폰\nSSG.COM\n상품쿠폰\n적용몰 : 신세계몰, 이마트몰\n적용 카테고리 : 뷰티 전체, 일상 헤어케어&바디케어\n1%\n선착순\n3만장\nSSG.COM\n2만원 이상 상품 구매시 최대 5천원 할인\nse\n쿠폰 다운로드 받기\n이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외-'

In [66]:
from google.cloud import vision

client = vision.ImageAnnotatorClient()
image = vision.Image()
image.source.image_uri = img_tmp[0]

response = client.text_detection(image=image)
texts = response.text_annotations

In [72]:

print(texts[0].description)

럭셔리뷰티 ~12% 상품쿠폰
적용몰 : 신세계백화점몰, 시코르
적용 카테고리 : 스킨케어, 메이크업, 헤어/바디, 향수
상품쿠폰
12%
선착순
5만장
4만원 이상 상품 구매시 최대 1만 5천원 할인
S P
쿠폰 다운로드 받기
-
- 이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외.
트렌드뷰티 ~7% 상품쿠폰
SSG.COM
상품쿠폰
적용몰 : 신세계몰, 이마트몰
적용 카테고리 : 뷰티 전체, 일상 헤어케어&바디케어
1%
선착순
3만장
SSG.COM
2만원 이상 상품 구매시 최대 5천원 할인
se
쿠폰 다운로드 받기
이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외-


In [75]:
info = texts[0].description.split("\n")

In [76]:
info

['럭셔리뷰티 ~12% 상품쿠폰',
 '적용몰 : 신세계백화점몰, 시코르',
 '적용 카테고리 : 스킨케어, 메이크업, 헤어/바디, 향수',
 '상품쿠폰',
 '12%',
 '선착순',
 '5만장',
 '4만원 이상 상품 구매시 최대 1만 5천원 할인',
 'S P',
 '쿠폰 다운로드 받기',
 '-',
 '- 이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외.',
 '트렌드뷰티 ~7% 상품쿠폰',
 'SSG.COM',
 '상품쿠폰',
 '적용몰 : 신세계몰, 이마트몰',
 '적용 카테고리 : 뷰티 전체, 일상 헤어케어&바디케어',
 '1%',
 '선착순',
 '3만장',
 'SSG.COM',
 '2만원 이상 상품 구매시 최대 5천원 할인',
 'se',
 '쿠폰 다운로드 받기',
 '이벤트 기간 내 ID 당 각 1장 다운로드/일부 특가상품 적용 제외-']